**scraping**

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pymysql

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.koreabaseball.com/TeamRank/TeamRank.aspx")

In [3]:
table = driver.find_element_by_css_selector('table.tData')
print(table.get_attribute('innerText'))

순위	팀명	경기	승	패	무	승률	게임차	최근10경기	연속	홈	방문
1	SSG	105	71	31	3	0.696	0	7승0무3패	2승	39-0-13	32-3-18
2	LG	101	61	39	1	0.610	9	7승0무3패	1승	27-0-23	34-1-16
3	키움	107	61	44	2	0.581	11.5	3승0무7패	1패	28-1-24	33-1-20
4	KT	103	56	45	2	0.554	14.5	6승0무4패	4승	28-1-24	28-1-21
5	KIA	102	50	51	1	0.495	20.5	3승0무7패	1패	24-0-27	26-1-24
6	롯데	105	45	56	4	0.446	25.5	5승0무5패	1승	18-3-32	27-1-24
7	두산	101	44	55	2	0.444	25.5	4승0무6패	2패	20-1-32	24-1-23
8	NC	100	43	54	3	0.443	25.5	6승1무3패	1승	22-3-24	21-0-30
9	삼성	104	42	60	2	0.412	29	4승0무6패	3패	18-2-31	24-0-29
10	한화	102	31	69	2	0.310	39	4승0무6패	3패	19-0-31	12-2-38


In [4]:
# th 명 가져오기
th_list = []
for i in range(1,13):
    ths = driver.find_element_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/table/thead/tr/th[{}]'.format(i))
    th_list.append(ths.get_attribute('innerText'))
    
print(th_list)   
print(th_list[0])
print(len(th_list))

['순위', '팀명', '경기', '승', '패', '무', '승률', '게임차', '최근10경기', '연속', '홈', '방문']
순위
12


In [5]:
# td 가져오기

td_list = []
for k in range(1,13):
    tds = driver.find_elements_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/table/tbody/tr[{}]'.format(k))
    for td_text in tds:
        td_list.append(td_text.text)
        td_elmts = td_list[k-1].split()
        print(td_elmts)

['1', 'SSG', '105', '71', '31', '3', '0.696', '0', '7승0무3패', '2승', '39-0-13', '32-3-18']
['2', 'LG', '101', '61', '39', '1', '0.610', '9', '7승0무3패', '1승', '27-0-23', '34-1-16']
['3', '키움', '107', '61', '44', '2', '0.581', '11.5', '3승0무7패', '1패', '28-1-24', '33-1-20']
['4', 'KT', '103', '56', '45', '2', '0.554', '14.5', '6승0무4패', '4승', '28-1-24', '28-1-21']
['5', 'KIA', '102', '50', '51', '1', '0.495', '20.5', '3승0무7패', '1패', '24-0-27', '26-1-24']
['6', '롯데', '105', '45', '56', '4', '0.446', '25.5', '5승0무5패', '1승', '18-3-32', '27-1-24']
['7', '두산', '101', '44', '55', '2', '0.444', '25.5', '4승0무6패', '2패', '20-1-32', '24-1-23']
['8', 'NC', '100', '43', '54', '3', '0.443', '25.5', '6승1무3패', '1승', '22-3-24', '21-0-30']
['9', '삼성', '104', '42', '60', '2', '0.412', '29', '4승0무6패', '3패', '18-2-31', '24-0-29']
['10', '한화', '102', '31', '69', '2', '0.310', '39', '4승0무6패', '3패', '19-0-31', '12-2-38']


**SQL**

In [6]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='3064', db='mywork', charset='utf8')
cursor = db.cursor()

In [17]:
# col 생성
sql = """
    create table KBO_team_rank (
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50),
        {} varchar(50)
    );
""".format(
        th_list[0], th_list[1], th_list[2], th_list[3],th_list[4], th_list[5], 
        th_list[6], th_list[7], th_list[8], th_list[9], th_list[10], th_list[11]
          )

cursor.execute(sql)
db.commit()

In [21]:
# 내용 추가
td_list = []
for k in range(1,13):
    tds = driver.find_elements_by_xpath('//*[@id="cphContents_cphContents_cphContents_udpRecord"]/table/tbody/tr[{}]'.format(k))
    val = []
    for td_text in tds:
        td_list.append(td_text.text)
        td_elmts = td_list[k-1].split()
        
        for rank_info in td_elmts:
            val.append(rank_info)
            sql = "INSERT INTO KBO_team_rank VALUES({});".format(val)
            rank_data = sql.replace('[', '', 1)
            rank_data = rank_data.replace(']', '', 1)

        cursor.execute(rank_data)
        
db.commit()